In [1]:
%env BASE_DIR=/home/jupyter/snippets

env: BASE_DIR=/home/jupyter/snippets


In [2]:
!gsutil cp gs://cs327e-open-access/fs_shopify.zip .

Copying gs://cs327e-open-access/fs_shopify.zip...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      


In [3]:
!gsutil cp gs://cs327e-open-access/fs_samples.zip .

Copying gs://cs327e-open-access/fs_samples.zip...
/ [1 files][  4.5 KiB/  4.5 KiB]                                                
Operation completed over 1 objects/4.5 KiB.                                      


In [4]:
!unzip $BASE_DIR/fs_shopify.zip

Archive:  /home/jupyter/snippets/fs_shopify.zip
   creating: fs_shopify/
  inflating: fs_shopify/apps.csv     
  inflating: fs_shopify/reviews.csv  
  inflating: fs_shopify/apps_categories.csv  
  inflating: fs_shopify/key_benefits.csv  
  inflating: fs_shopify/categories.csv  
  inflating: fs_shopify/pricing_plans.csv  
  inflating: fs_shopify/pricing_plan_features.csv  


In [5]:
!unzip $BASE_DIR/fs_samples.zip

Archive:  /home/jupyter/snippets/fs_samples.zip
   creating: fs_samples/
  inflating: fs_samples/delete_single.py  
  inflating: fs_samples/write_batch.py  
  inflating: fs_samples/read_nested.py  
  inflating: fs_samples/read_count.py  
  inflating: fs_samples/write_single.py  
  inflating: fs_samples/read_where.py  
  inflating: fs_samples/write_nested.py  
  inflating: fs_samples/delete_field.py  
  inflating: fs_samples/read_single.py  
  inflating: fs_samples/update_single.py  
  inflating: fs_samples/read_doc_with_filter2.py  


In [6]:
import pandas as pd
from google.cloud import firestore

# Initialize Firestore client
db = firestore.Client()

df = pd.read_csv('fs_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
app_rows = df.values.tolist()

# Define a reference to the root collection "Apps"
apps_collection_ref = db.collection("apps")

# Define the structure for the "Apps" collection document

for app_row in app_rows:
    app_data = {}
    app_data['id'] = app_row[0]
    app_data['url'] = app_row[1]
    app_data['title'] = app_row[2]
    app_data['developer'] = app_row[3]
    app_data['developer_link'] = app_row[4]
    app_data['icon'] = app_row[5]
    app_data['rating'] = app_row[6]
    app_data['reviews_count'] = app_row[7]
    

# Create a document in the "Apps" collection with the specified data
apps_collection_ref.add(app_data)

df = pd.read_csv('fs_shopify/categories.csv', sep=',', header=0, lineterminator='\n')
category_rows = df.values.tolist()

# Define a reference to the "Categories" subcollection for the created "Apps" document
categories_subcollection_ref = apps_collection_ref.document(app_row[0]).collection("categories")

# Define data for the "Categories" subcollection
for category_row in category_rows:
    category_data = {}
    category_data['id'] = category_row[0]
    category_data['title'] = category_row[1]

# Create a document in the "Categories" subcollection
categories_subcollection_ref.add(category_data)

df = pd.read_csv('fs_shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
keyben_rows = df.values.tolist()

keyben_collection_ref = db.collection("key_benefits")

# Define the structure for the "Apps" collection document

for keyben_row in keyben_rows:
    keyben_data = {}
    keyben_data['app_id'] = keyben_row[0]
    keyben_data['title'] = keyben_row[1]
    keyben_data['description'] = keyben_row[2]
    

keyben_collection_ref.add(keyben_data)


apps_subcollection_ref = keyben_collection_ref.document(keyben_row[0]).collection("apps")


apps_subcollection_ref.add(app_data)

df = pd.read_csv('fs_shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
ppf_rows = df.values.tolist()


ppf_collection_ref = db.collection("pricing_plan_features")


for ppf_row in ppf_rows:
    ppf_data = {}
    ppf_data['app_id'] = ppf_row[0]
    ppf_data['pricing_plan_id'] = ppf_row[1]
    ppf_data['feature'] = ppf_row[2]
    

ppf_collection_ref.add(ppf_data)

apps_subcollection_ref = ppf_collection_ref.document(ppf_row[0]).collection("apps")


apps_subcollection_ref.add(app_data)


(DatetimeWithNanoseconds(2023, 10, 20, 3, 57, 57, 892778, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f37f066cd90>)

In [10]:
from google.cloud import firestore

# Initialize Firestore client
db = firestore.Client()

# Define the collection names
collection_names = ["apps", "categories", "pricing_plans_features", "key_benefits"]

# Create a dictionary to store the document counts for each collection
document_counts = {}

# Loop through the collection names and count the documents in each collection
for collection_name in collection_names:
    collection_ref = db.collection(collection_name)
    collection_count = collection_ref.stream()  # Stream all documents in the collection
    document_counts[collection_name] = len(list(collection_count))

# Print the document counts for each collection
for collection_name, count in document_counts.items():
    print(f"Collection: {collection_name}, Document Count: {count}")

Collection: apps, Document Count: 1
Collection: categories, Document Count: 0
Collection: pricing_plans_features, Document Count: 0
Collection: key_benefits, Document Count: 1


In [31]:
from google.cloud import firestore

# Initialize Firestore client
db = firestore.Client()

# Define the query parameters
category_title = "Productivity"
limit = 10

# Create a query to filter apps by category
query = db.collection("apps").where("categories.title", "==", "Productivity")

# Order the results by rating in descending order
query = query.order_by("rating", direction=firestore.Query.DESCENDING)

# Limit the query results to the first 10 records
query = query.limit(limit)

# Execute the query
results = query.stream()

# Iterate through the query results and print the desired fields
for app in results:
    app_data = app.to_dict()
    print("App ID:", app_data["id"])
    print("Title:", app_data["title"])
    print("Developer:", app_data["developer"])
    print("Rating:", app_data["rating"])
    print("Reviews Count:", app_data["reviews_count"])
    print("\n")


FailedPrecondition: 400 The query requires an index. You can create it here: https://console.firebase.google.com/v1/r/project/unified-canyon-397523/firestore/indexes?create_composite=ClJwcm9qZWN0cy91bmlmaWVkLWNhbnlvbi0zOTc1MjMvZGF0YWJhc2VzLyhkZWZhdWx0KS9jb2xsZWN0aW9uR3JvdXBzL2FwcHMvaW5kZXhlcy9fEAEaFAoQY2F0ZWdvcmllcy50aXRsZRABGgoKBnJhdGluZxACGgwKCF9fbmFtZV9fEAI

In [36]:
from google.cloud import firestore

# Initialize Firestore client
db = firestore.Client()

# Define the query parameters
limit = 10

# Create a query to retrieve the top 10 apps with the highest number of reviews
query = db.collection("apps")

# Order the results by reviews_count in descending order
query = query.order_by("reviews_count", direction=firestore.Query.DESCENDING)

# Limit the query results to the first 10 records
query = query.limit(limit)

# Execute the query
results = query.stream()

# Iterate through the query results and print the desired fields
for app in results:
    app_data = app.to_dict()
    print("App ID:", app_data["id"])
    print("Title:", app_data["title"])
    print("Developer:", app_data["developer"])
    print("Rating:", app_data["rating"])
    print("Reviews Count:", app_data["reviews_count"])
    print("\n")

App ID: c71b9941-fa61-4e97-acb6-b632a2a175b4
Title: Feedo
Developer: Regalix
Rating: 4.2
Reviews Count: 36


